In [3]:
import pandas as pd
import numpy as np

from route import Route
from weather_forecast import WeatherForecast

current_position = {'longitude': 130.868566,
                    'latitude': -12.432466}
delta_spacing = 100000.0 # in meters
number_sites = 3

In [4]:
route = Route()
# route.get_final_data(current_position)
# route.get_final_data(current_position, delta_spacing=delta_spacing)
dataframe = route.get_final_data(current_position, number_sites=number_sites)

    longitude   latitude    altitude   cumDistance  gradient  maxSpeed
0  130.868566 -12.432466   11.500000  0.000000e+00  0.024056       100
1  133.803762 -23.861845  557.690488  1.513018e+06  0.004450       130
2  138.608989 -34.935232   53.250000  3.026035e+06 -0.017856        50


In [7]:
api = WeatherForecast()
api.add_sites(dataframe)

Response status from siteinfo: OK.
Current sites:
Empty DataFrame
Columns: [name, site_id, longitude, latitude]
Index: []
Response status from siteadd: OK.
Site with name 0 has been added.
Response status from siteadd: OK.
Site with name 1 has been added.
Response status from siteadd: OK.
Site with name 2 has been added.
All sites have been added.
  name site_id  longitude  latitude
0    0  584867    130.869  -12.4325
1    1  584868    133.804  -23.8618
2    2  584869    138.609  -34.9352


In [ ]:
api.get_site_edit(584867, True, name="cc")
api.get_site_edit(584868, True, name="cc", position=)
api.get_site_edit(584869, True, position=)

In [6]:
api.delete_all_sites()

Response status from sitedelete: OK (200).
Site with id 584858 has been removed.
Response status from sitedelete: OK (200).
Site with id 584859 has been removed.
Response status from sitedelete: OK (200).
Site with id 584860 has been removed.
Response status from sitedelete: OK (200).
Site with id 584861 has been removed.
Response status from sitedelete: OK (200).
Site with id 584862 has been removed.
Response status from sitedelete: OK (200).
Site with id 584863 has been removed.
Response status from sitedelete: OK (200).
Site with id 584864 has been removed.
Response status from sitedelete: OK (200).
Site with id 584865 has been removed.
Response status from sitedelete: OK (200).
Site with id 584866 has been removed.
All sites have been deleted.
Empty DataFrame
Columns: [name, site_id, longitude, latitude]
Index: []
